In [1]:
# To get multiple outputs from one code cell (without using print()):
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image, Audio
import sys
from pathlib import Path
import pandas as pd
from pprint import pprint as pp

# For documenting the current environment:
def sys_info():
    frmt = '\nPython ver: {}\nPython env: {}\n'
    frmt += 'OS:         {}\nCurrent dir: {}\n'
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))

# For enabling imports from current project code:
def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = F'Path already in sys.path: {newp}'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = F'Path added to sys.path: {newp}'
    print(msg)

# If this ipynb file is inside a folder, eg ./notebooks, 
# the project code is assumed to reside 1 level up:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), up=Path.cwd().name.startswith(nb_folder))

# For py modules/methods discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
       Example:
       >filter_dir(re) # lists the public methods of the re module.
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

# To create often-used subfolders:
def get_project_dirs(which=['data', 'images'],
                     use_parent=True):
    '''Create folder(s) named in `which` at the ipynb parent level.'''
    if use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    dir_lst = []    
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

# For adding colorfull divider in the nb:
def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)


def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:3em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')

# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2
# .................

Path added to sys.path: C:/Users/catch/Documents/GitHub/DU-event-transcript-demo/resources/EventManagement


In [6]:
import ipywidgets as ipw
from collections import OrderedDict

from manage import (EventMeta as Meta,
                    EventTranscription as TRX,
                    Workflow as FLO,
                    Controls as CTR,
                    Utils as UTL)

main_readme = Meta.MAIN_README

In [27]:
main_readme

WindowsPath('C:/Users/catch/Documents/GitHub/DU-event-transcript-demo/README_new.md')

In [56]:
del ap

In [214]:
class AppControls:
    def __init__(self):
        self.actions = OrderedDict([('Add an event',
                                     ['Enter Info','Validate','Save','Show Readme', 'Show File']),
                                    ('Modify an event',
                                     ['Modify Event','Validate','Save','Show Readme', 'Show File']),
                                    ('Edit a transcript',
                                     ['Edit Transcript','Save','Show Readme', 'Show File'])
                                   ])
        #self.TR = None
        self.PC = None # Controls.PageControl instance
        
        # output controls first:
        self.info_out = ipw.Output() # == right-sidebar
        
        self.left_sidebar = self.get_left()
        self.center = self.get_center()
        self.left_sidebar.observe(self.menu_selection, 'value')
        
        self.center.observe(self.info_display, 'selected_index')
        self.dl1 = ipw.dlink((self.left_sidebar, 'selected_index'),
                             (self.center, 'selected_index'))

        self.page = ipw.AppLayout(header=self.get_app_hdr(),
                                  left_sidebar=self.left_sidebar,
                                  center=self.center,
                                  right_sidebar=self.info_out,
                                  footer=None,
                                  pane_widths=[1, 5, 1],
                                  pane_heights=[1, 3, 1],
                                  #justify_content='space-between'
                                  )
        #setattr(self.page, 'user_dict', None)
        setattr(self.page, 'data_dict', None)


    def get_app_hdr(self):
        style = "text-align:center;padding:5px;background:#c2d3ef;"
        style += "color:#ffffff;font-size:3em;"
        style += "width:100%,height=50%"
        div = F' <div style="{style}">Data Umbrella Event Management</div>'
        hdr_html = FLO.show_du_logo_hdr(as_html=False) + div
        return ipw.HTML(hdr_html)


    def get_center(self):
        """ Create 3 Tabs for actions."""
        lo_tabs = ipw.Layout(display='flex',
                             flex_flow='column',
                             align_items='stretch',
                             justify_content='center',
                             width='98%')
        tabs = ipw.Tab(selected_index=None, layout=lo_tabs)
        # Tabs from menu action:
        ks = list(self.actions.keys()) + ['readme', 'file']
        # 1st child: message output
        tabs.children = [ipw.VBox([ipw.Output()]) for k in ks]
        for i, k in enumerate(ks):
            tabs.set_title(i, k.split()[0].upper())
        return tabs

    
    def get_left(self):
        """
        Create Accordion MENU with VBox for actions.keys as children.
        Populate Accordion MENU with ToggleButtons children that have:
         - a 'parent_idx' attribute
         - observe function for ToggleButtons children
        """
        acc_items = [ipw.VBox(description=k) for k in list(self.actions.keys())]
        menu_acc = ipw.Accordion(children=acc_items,
                                 selected_index=None)
        for i, (k, v) in enumerate(self.actions.items()):
            btn = ipw.ToggleButtons(options=v, value=None,
                                    button_style='info')
            setattr(btn, 'parent_idx', i)
            btn.observe(self.menu_tog_sel, names='value')
            menu_acc.children[i].children = [btn]
            menu_acc.set_title(i, k.upper())
        return menu_acc
    
    
    # left_sidebar.observe
    def menu_selection(change):
        """ 
        Synchronized center (Tabs) selection with 
        left sidebar (menu) selection.
        """
        self.center.selected_index = self.left_sidebar.selected_index


    # center.observe
    def info_display(self, change):
        """Right side panel: info about selected op."""
        self.page.right_sidebar.clear_output()
        wgt = change['owner']
        # Link tab selection index with info panel:
        if wgt.selected_index is None:
            t = 3
        else:
            t = wgt.selected_index
        if t > 3:
            return
        event_fn = {0:'ADD', 1:'MODIFY', 2:'EDIT', 3:'INIT'}
        which = FLO.EventFunction[event_fn[t]]
        EF = FLO.DisplaySectionInfo(which)
        info_val = EF.show_section_info()
        with self.page.right_sidebar:
            display(info_val)
   

    def msg_out(self, idx, msg):
        self.center.children[idx].children[0].clear_output()
        with self.center.children[idx].children[0]:
            display(msg)

    
    def validate(self, idx):
        self.center.children[idx].children[0].clear_output()
        
        input_form = self.PC.page.children[1]
        with self.center.children[idx].children[0]:
            try:
                self.page.data_dict = FLO.validate_form_entries(input_form,
                                                           self.PC.page.user_dict,
                                                           self.PC.TR)
                print('Validated!')
            except:
                print('Validation Error: Fix & Try again.')
                
    
    def save_entry(self, idx):
        self.center.children[idx].children[0].clear_output()
        
        with self.center.children[idx].children[0]:
            if self.page.data_dict is None:
                print('Validate first!')
                return
            if self.PC.TR is None:
                print('TR object not instanciated.')
                return
            try:
                self.PC.TR.update_dict(self.page.data_dict)
                print('Update dict: OK!')
            except:
                print('Update dict: Something went wrong.')
                return
            try:
                self.PC.TR.update_readme()
                print('Update readme: OK!')
            except:
                print('Update readme: Something went wrong.')
                return
            try:
                self.PC.TR.save_transcript_md()
                print('Save: OK!')
                #if idx == 1:
                #    self.PC.page.children[0].children[1].children[1].disabled = False
            except:
                if idx == 0:
                    msg = 'Save starter transcript: Something went wrong.'
                else:
                    msg = 'Save transcript: Something went wrong.'
                print(msg)


    def save_edit(self, idx):
        """ Save edited transcript. """
        self.center.children[idx].children[0].clear_output()

        with self.center.children[idx].children[0]:
            if self.PC.transcriber_txt.value == '?':
                print("'?' is not a good name!")
            try:
                upd = self.PC.initial_status != self.PC.status_sel.value
                upd = upd or (self.PC.initial_transcriber != self.PC.transcriber_txt.value)
                if upd:
                    self.PC.TR.event_dict['status'] = self.PC.status_sel.value
                    self.PC.TR.event_dict['transcriber'] = self.transcriber_txt.value
                    self.PC.TR.update_readme()
                    print('Updated README.')
            except:
                print('Could not update README.')
                return
            try:
                #self.PC.TR.insert_md_transcript(new_trx=self.PC.editarea.value)
                self.PC.TR.save_transcript_md(new_trx=self.PC.editarea.value)
                self.PC.page.children[0].children[1].children[3].children[0].disabled = False
                self.PC.editarea.value = ''
                print('Updated Event file.')
            except:
                print('Could not update Event file.')

        
    def close_file_tabs(self):
        tabs = len(self.center.children)
        if tabs > 3:
            for t in range(3,5):
                self.center.children[t].close()


    def add_file_tab0(self, idx):
        tabs = len(self.center.children)
        if tabs < 5:
            if tabs < idx:
                self.center.children += (ipw.VBox([ipw.Output(), ipw.Output()]),)
                self.center.set_title(3, 'README')
                tabs = len(self.center.children)

            if tabs == idx:
                title = 'README' if idx == 3 else 'FILE'
                self.center.children += (ipw.VBox([ipw.Output(), ipw.Output()]),)
                self.center.set_title(idx, title)


    def show_mdfile(self, iparent, idx):
        """Add tab for Md file. Here, idx = tab index"""
        #self.add_file_tab(idx)
        # Add 2nd Output for the file:
        if len(self.center.children[idx].children) == 1:
            self.center.children[idx].children += (ipw.Output(),)
        
        self.center.children[idx].children[1].clear_output()
        if idx == 3:
            with self.center.children[3].children[1]:
                Meta.show_md_file(main_readme)
            self.center.selected_index = 3
        else:
            self.center.selected_index = 4
            #input_form = self.center.children[iparent].children[1]
            #self.TR = input_form.TR
            if self.PC.TR is not None:
                yr = self.PC.TR.event_dict['year']
                fname = self.PC.TR.event_dict['transcript_md']
                mdfile = Meta.REPO_PATH.joinpath(yr, fname)
                if mdfile.exists():
                    with self.center.children[4].children[1]:
                        Meta.show_md_file(mdfile, kind='Transcript')
                else:
                    self.msg_out(4, F'File not found: {mdfile}.')
            else:
                self.msg_out(4, 'PageControls.TR object not instanciated.')


    def menu_tog_sel(self, change):
        wgt = change['owner']
        iparent = wgt.parent_idx
        tog_val = wgt.value

        if iparent == 0:
            #self.close_file_tabs()
            #self.TR = None #reset?
            if tog_val == 'Enter Info':
                self.PC = CTR.PageControls(0)
                entry_group = self.PC.page
                # Add 2nd control in tab vbox == input_form:
                self.center.children[0].children += (entry_group,)
                
            elif tog_val == 'Validate':
                self.validate(iparent)
            elif tog_val == 'Save':
                self.save_entry(iparent)
            elif tog_val == 'Show Readme':
                self.show_mdfile(iparent, 3)
            elif tog_val == 'Show File':
                self.show_mdfile(iparent, 4)

        elif iparent == 1:
            #self.close_file_tabs()
            if tog_val == 'Modify Event':
                self.PC = CTR.PageControls(1)
                edit_page = self.PC.page
                # Add 2nd control in tab vbox == input_form:
                self.center.children[1].children += (edit_page, )
            elif tog_val == 'Validate':
                self.validate(iparent)
            elif tog_val == 'Save':
                self.save_entry(iparent)
            elif tog_val == 'Show Readme':
                self.show_mdfile(iparent, 3)
            elif tog_val == 'Show File':
                self.show_mdfile(iparent, 4)

        elif iparent == 2:
            #self.close_file_tabs()
            if tog_val == 'Edit Transcript':
                self.PC = CTR.PageControls(2)
                edit_page = self.PC.page
                # Add 2nd control in tab vbox == input_form:
                self.center.children[2].children += (edit_page, )
            elif tog_val == 'Save':
                self.save_edit(iparent)
            elif tog_val == 'Show Readme':
                self.show_mdfile(iparent, 3)
            elif tog_val == 'Show File':
                self.show_mdfile(iparent, 4)


    def __repr__(self):
        from pprint import pformat
        return pformat(self.__dict__)
    

In [215]:
ap = AppControls()
ap.page

AppLayout(children=(HTML(value='<div style="text-align:center;padding:5px;width:98%"><a href="https://www.data…

In [209]:
ap.PC.page.children[0].children[1].children[3].children[0].disabled
ap.center.children[2].children[1].children[0].children[1].children[3].children[0].disabled

True

True

In [176]:
data_dict = FLO.validate_form_entries(ap.PC.page.children[1],
                                      ap.page.user_dict,
                                      ap.PC.TR)

IndexError: tuple index out of range

In [154]:
kids = len(ap.page.children)
kids
for k in range(kids):
    k, type(ap.page.children[k])

4

(0, ipywidgets.widgets.widget_string.HTML)

(1, ipywidgets.widgets.widget_selectioncontainer.Accordion)

(2, ipywidgets.widgets.widget_output.Output)

(3, ipywidgets.widgets.widget_selectioncontainer.Tab)

In [147]:
type(ap.page.center.children[2].children[0])
type(ap.page.center.children[2].children[1]), len(ap.page.center.children[2].children[1].children)

ipywidgets.widgets.widget_output.Output

(ipywidgets.widgets.widget_box.VBox, 3)

In [148]:
type(ap.page.center.children[2].children[1].children[0])
type(ap.page.center.children[2].children[1].children[1])
type(ap.page.center.children[2].children[1].children[2])

ipywidgets.widgets.widget_box.VBox

ipywidgets.widgets.widget_string.HTML

ipywidgets.widgets.widget_string.Textarea

In [126]:
input_form = ap.center.children[2].children[1].children[1]
type(input_form )

ipywidgets.widgets.widget_string.HTML

In [127]:
ap.PC.status_sel.value

'Not yet processed (editor needed)'

In [114]:
ap.PC.TR.event_dict['year'], ap.PC.TR.event_dict['idn']
ap.PC.TR.event_dict['status'], ap.PC.TR.event_dict['has_transcript']

('2021', '21')

('Not yet processed (editor needed)', True)

In [117]:
ap.PC.TR.event_dict['audio_track']
ap.PC.TR.event_dict['video_embed']

WindowsPath('C:/Users/catch/Documents/GitHub/DU-event-transcript-demo/resources/EventManagement/data/meta/2021_21_MHAjCcBfT_A.mp4')

'\n<iframe width="560" height="315" \n        src="https://www.youtube-nocookie.com/embed/MHAjCcBfT_A?cc_load_policy=1&autoplay=0" \n        frameborder="0">\n</iframe>\n'

In [134]:
type(ap.center.children[2].children[1])

ipywidgets.widgets.widget_box.VBox

In [64]:
i = 4
len(ap.page.center.children[i].children)
type(ap.page.center.children[i].children[0])
type(ap.page.center.children[i].children[1])

2

ipywidgets.widgets.widget_output.Output

ipywidgets.widgets.widget_output.Output

In [65]:
display(ap.page.center.children[i].children[1])

Output()

In [47]:
ap.page.center.children += (ipw.VBox([ipw.Output(), ipw.Output()]),)
ap.page.center.selected_index = 3

In [73]:
len(ap.page.center.children[1].children)

1

In [68]:
ap.show_mdfile(3)

In [26]:
type(ap.page.left_sidebar)
ap.page.left_sidebar.selected_index

ipywidgets.widgets.widget_selectioncontainer.Accordion

2

In [69]:
i = ap.page.center.selected_index
i
type(ap.page.center.children[i])
kids = len(ap.page.center.children[i].children)
print('kids:', kids)
type(ap.page.center.children[i].children[0])
if kids > 1:
    type(ap.page.center.children[i].children[1])

3

ipywidgets.widgets.widget_box.VBox

kids: 2


ipywidgets.widgets.widget_output.Output

ipywidgets.widgets.widget_output.Output

In [16]:
len(ap.page.data_dict.keys())
ap.TR.event_dict['idn']

26

'22'

In [13]:
if ap.TR is not None:
    ap.save_entry(0)
else:
    self.msg_out(0, 'TR object not instanciated.')

In [15]:

#ap.TR.update_dict(ap.page.data_dict)
#ap.TR.update_readme()
#ap.TR.save_transcript_md()

'22'

In [17]:
if len(ap.page.center.children[3].children) == 1:
    # Add an Output as 1st vbx item:
    ap.page.center.children[3].children += (ipw.Output(), )
    
with ap.page.center.children[3].children[1]:
        Meta.show_md_file(main_readme)
    

IndexError: tuple index out of range

# idn selection box:

In [99]:
df, empty_last_row, delims = Meta.df_from_readme_tbl()
df.tail(4)

,N,Speaker,Talk Transcript,Transcriber,Status,Notes,year,name
22,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.
23,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.
24,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.
25,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.


In [130]:
edit_page = get_edit_page()
edit_page

In [136]:
du_gui

AppLayout(children=(HTML(value='<div style="text-align:center;padding:5px;width:98%"><a href="https://www.data…

In [55]:
form_data_dict = FLO.validate_form_entries(new_entry_group, new_input, TR)
form_data_dict

OrderedDict([('presenter', 'cat chenal'),
             ('title', 'my presentation title'),
             ('event_url', 'N.A.'),
             ('yt_video_id', 'MHAjCcBfT_A'),
             ('slides_url', 'N.A.'),
             ('repo_url', 'https://github.com/CatChenal'),
             ('notebook_url', 'N.A.'),
             ('transcriber', 'N.A.'),
             ('extra_references',
              "- Binder: each listed item should have a 'list header', e.g. '- Binder'  \n- Twitter: Use this format: [full name 1](twitter url), etc.     \n- Wiki: This is an excellent [wiki on transcription](http://en.wikipedia.org/wiki/Main_Page) "),
             ('video_href', 'N.A.'),
             ('video_href_src', 'N.A.'),
             ('video_href_alt', 'N.A.'),
             ('video_href_w', '25%'),
             ('formatted_transcript', 'N.A.'),
             ('year', '2020'),
             ('idn', '22'),
             ('video_url', 'https://www.youtube.com/watch?v=MHAjCcBfT_A'),
             ('title_kw', 'fo

---
---